In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### data.json ahs been already created as well as the config.yml file

## Training the NLU Model

In [2]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


In [23]:
# from rasa_nlu.training_data import load_data
# from rasa_nlu.config import RasaNLUModelConfig
# from rasa_nlu.model import Trainer
# from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("../data/data.json")

# trainer to educate our pipeline
trainer = Trainer(config.load("../config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../data/data.json is rasa_nlu
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 887 (3 distinct intents)
	- Found intents: 'sales', 'greet', 'goodbye'
	- entity examples: 878 (3 distinct entities)
	- found entities: 'date', 'store_id', 'sales_entity'

INFO:rasa_nlu.model:Starting to train component tokenizer_whitespace
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_entity_featurizer_regex
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_synonyms
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_count_vectors
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_classifie

In [21]:
pprint((interpreter.parse("calculate my cummulative sales in 2014")))

{
  "intent": {
    "name": "sales",
    "confidence": 0.9539963603019714
  },
  "entities": [
    {
      "start": 10,
      "end": 12,
      "value": "my",
      "entity": "store_id",
      "confidence": 0.9979437656331461,
      "extractor": "ner_crf"
    },
    {
      "start": 13,
      "end": 30,
      "value": "cummulative sales",
      "entity": "sales_entity",
      "confidence": 0.8946741200976823,
      "extractor": "ner_crf"
    },
    {
      "start": 34,
      "end": 38,
      "value": "2014",
      "entity": "date",
      "confidence": 0.9981110891988527,
      "extractor": "ner_crf"
    },
    {
      "start": 31,
      "end": 38,
      "text": "in 2014",
      "value": "2014-01-01T00:00:00.000-08:00",
      "confidence": 1.0,
      "additional_info": {
        "values": [
          {
            "value": "2014-01-01T00:00:00.000-08:00",
            "grain": "year",
            "type": "value"
          }
        ],
        "value": "2014-01-01T00:00:00.000-08:00",
    

In [ ]:
temp_dict = {
  "intent": {
    "name": "sales",
    "confidence": 0.9576094746589661
  },
  "entities": [
    {
      "start": 19,
      "end": 43,
      "value": "01-01-2017 to 01-02-2018",
      "entity": "date",
      "confidence": 0.5087736726201181,
      "extractor": "ner_crf"
    },
    {
      "start": 14,
      "end": 43,
      "text": "from 01-01-2017 to 01-02-2018",
      "value": {
        "to": "2018-01-03T00:00:00.000-08:00",
        "from": "2017-01-01T00:00:00.000-08:00"
      },
      "confidence": 1.0,
      "additional_info": {
        "values": [
          {
            "to": {
              "value": "2018-01-03T00:00:00.000-08:00",
              "grain": "day"
            },
            "from": {
              "value": "2017-01-01T00:00:00.000-08:00",
              "grain": "day"
            },
            "type": "interval"
          }
        ],
        "to": {
          "value": "2018-01-03T00:00:00.000-08:00",
          "grain": "day"
        },
        "from": {
          "value": "2017-01-01T00:00:00.000-08:00",
          "grain": "day"
        },
        "type": "interval"
      },
      "entity": "time",
      "extractor": "ner_duckling_http"
    }
  ],
  "intent_ranking": [
    {
      "name": "sales",
      "confidence": 0.9576094746589661
    },
    {
      "name": "goodbye",
      "confidence": 0.0
    },
    {
      "name": "greet",
      "confidence": 0.0
    }
  ],
  "text": "What is sales from 01-01-2017 to 01-02-2018"
}

In [ ]:
temp_dict['entities'][1]['additional_info']

# Custom Action

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet


class ActionSales(Action):
    def name(self):
        return 'action_sales'

    
    def run(self, dispatcher, tracker, domain):
        store_id = tracker.get_slot('store_id')
        date = tracker.get_slot('time')
        #d1 = date[:10]
        
        time_entity = next((e for e in tracker.latest_message.entities if
                                 e['entity'] == 'time'), None)

        import pyodbc
        
        import pandas as pd

        
        cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                	"Server=pdssoldb.database.windows.net;"
                        "Database=PDS_Sol_1;"
                        "uid=Sandeep;pwd=Abcd@1111")
        
        
        #query = "select sum(sales) from DH_ALL_STORES_ALL_DAYS_SALES_PRED_A where store_id = 369 and date = {}".format("'"+ d1 +"'")
        #df = pd.read_sql_query(query,cnxn)
        #response = """It is currently the default message, since the database connection is pending"""
        #response = pd.read_sql_query(query,cnxn)
        response = time_entity
        
        dispatcher.utter_message("Time entity is  {}".format(response))
		#return [SlotSet('store_id',store_id)]

In [7]:

from  rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
# fallback = FallbackPolicy(fallback_action_name="utter_unclear",
#                           core_threshold=0.2,
#                           nlu_threshold=0.1)

agent = Agent('../retail_domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])#, fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('../data/stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

INFO:apscheduler.scheduler:Scheduler started
Using TensorFlow backend.
/home/pds_gcloud/anaconda3/envs/rasa/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks:   0%|          | 0/6 [00:00<?, ?it/s, # trackers=1]ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
Processed Story Blocks: 100%|██████████| 6/6 [00:00<00:00, 365.50it/s, # trackers=1]
ERROR:rasa_core.trackers:Tried 

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
Processed Story Blocks:   0%|          | 0/6 [00:00<?, ?it/s, # trackers=19]ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added 

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
Processed Story Blocks: 100%|██████████| 6/6 [00:00<00:00, 47.79it/s, # trackers=19]
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure y

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

...

329/329 [==============================] - 0s 174us/step - loss: 0.3373 - acc: 0.8024
Epoch 158/200
329/329 [==============================] - 0s 171us/step - loss: 0.3364 - acc: 0.7933
Epoch 159/200
329/329 [==============================] - 0s 183us/step - loss: 0.3315 - acc: 0.8146
Epoch 160/200
329/329 [==============================] - 0s 179us/step - loss: 0.3352 - acc: 0.7964
Epoch 161/200
329/329 [==============================] - 0s 177us/step - loss: 0.3209 - acc: 0.8237
Epoch 162/200
329/329 [==============================] - 0s 188us/step - loss: 0.3335 - acc: 0.7994
Epoch 163/200
329/329 [==============================] - 0s 177us/step - loss: 0.3254 - acc: 0.8085
Epoch 164/200
329/329 [==============================] - 0s 174us/step - loss: 0.3365 - acc: 0.8024
Epoch 165/200
329/329 [==============================] - 0s 168us/step - loss: 0.3431 - acc: 0.8116
Epoch 166/200
329/329 [==============================] - 0s 170us/step - loss: 0.3323 - acc: 0.8176
Epoch 167/200


INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all yo

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set non existent slot 'date'. Make sure you added all your slots to your domain file.
ERROR:rasa_core.trackers:Tried to set no

In [9]:
from rasa_core.agent import Agent
agent = Agent.load('./models/dialogue', interpreter=model_directory)

/home/pds_gcloud/anaconda3/envs/rasa/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
INFO:tensorflow:Restoring parameters from /home/pds_gcloud/chatbot/retail369/Notebook/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt


In [24]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
what is sales yesterday in store1
Time entity is  {'start': 14, 'end': 23, 'text': 'yesterday', 'value': '2018-08-22T00:00:00.000-07:00', 'confidence': 1.0, 'additional_info': {'values': [{'value': '2018-08-22T00:00:00.000-07:00', 'grain': 'day', 'type': 'value'}], 'value': '2018-08-22T00:00:00.000-07:00', 'grain': 'day', 'type': 'value'}, 'entity': 'time', 'extractor': 'ner_duckling_http'}
what is sales next week in stroe 1
Time entity is  {'start': 14, 'end': 23, 'text': 'next week', 'value': '2018-08-27T00:00:00.000-07:00', 'confidence': 1.0, 'additional_info': {'values': [{'value': '2018-08-27T00:00:00.000-07:00', 'grain': 'week', 'type': 'value'}], 'value': '2018-08-27T00:00:00.000-07:00', 'grain': 'week', 'type': 'value'}, 'entity': 'time', 'extractor': 'ner_duckling_http'}
what is sales last month 
Kindly provide the store id or the name of the store you are seeking the information!
in store1
Time entity is  None


what is sales in third quarter this year in store123
Time entity is  {'start': 17, 'end': 40, 'text': 'third quarter this year', 'value': '2018-07-01T00:00:00.000-07:00', 'confidence': 1.0, 'additional_info': {'values': [{'value': '2018-07-01T00:00:00.000-07:00', 'grain': 'quarter', 'type': 'value'}], 'value': '2018-07-01T00:00:00.000-07:00', 'grain': 'quarter', 'type': 'value'}, 'entity': 'time', 'extractor': 'ner_duckling_http'}
what is sale this year in store1
Time entity is  {'start': 13, 'end': 22, 'text': 'this year', 'value': '2018-01-01T00:00:00.000-08:00', 'confidence': 1.0, 'additional_info': {'values': [{'value': '2018-01-01T00:00:00.000-08:00', 'grain': 'year', 'type': 'value'}], 'value': '2018-01-01T00:00:00.000-08:00', 'grain': 'year', 'type': 'value'}, 'entity': 'time', 'extractor': 'ner_duckling_http'}
what sale last year i store1234
Time entity is  {'start': 10, 'end': 19, 'text': 'last year', 'value': '2017-01-01T00:00:00.000-08:00', 'confidence': 1.0, 'additional_inf

# Logic for date - custom action

In [25]:
# DAY 

day_dict = {  
   'start':24,
   'end':32,
   'text':'tomorrow',
   'value':'2018-08-24T00:00:00.000-07:00',
   'confidence':1.0,
   'additional_info':{  
      'values':[  
         {  
            'value':'2018-08-24T00:00:00.000-07:00',
            'grain':'day',
            'type':'value'
         }
      ],
      'value':'2018-08-24T00:00:00.000-07:00',
      'grain':'day',
      'type':'value'
   },
   'entity':'time',
   'extractor':'ner_duckling_http'
}

In [28]:
day_dict.keys()

dict_keys(['start', 'end', 'text', 'value', 'confidence', 'additional_info', 'entity', 'extractor'])

In [31]:
day_dict['additional_info'].keys()

dict_keys(['values', 'value', 'grain', 'type'])

In [32]:
# Interval Date
interval_dict = {  
   'start':13,
   'end':44,
   'text':'from january to march this year',
   'value':{  
      'to':'2018-04-01T00:00:00.000-07:00',
      'from':'2018-01-01T00:00:00.000-08:00'
   },
   'confidence':1.0,
   'additional_info':{  
      'values':[  
         {  
            'to':{  
               'value':'2018-04-01T00:00:00.000-07:00',
               'grain':'month'
            },
            'from':{  
               'value':'2018-01-01T00:00:00.000-08:00',
               'grain':'month'
            },
            'type':'interval'
         },
         {  
            'to':{  
               'value':'2018-04-01T00:00:00.000-07:00',
               'grain':'month'
            },
            'from':{  
               'value':'2017-01-01T00:00:00.000-08:00',
               'grain':'month'
            },
            'type':'interval'
         },
         {  
            'to':{  
               'value':'2018-04-01T00:00:00.000-07:00',
               'grain':'month'
            },
            'from':{  
               'value':'2016-01-01T00:00:00.000-08:00',
               'grain':'month'
            },
            'type':'interval'
         }
      ],
      'to':{  
         'value':'2018-04-01T00:00:00.000-07:00',
         'grain':'month'
      },
      'from':{  
         'value':'2018-01-01T00:00:00.000-08:00',
         'grain':'month'
      },
      'type':'interval'
   },
   'entity':'time',
   'extractor':'ner_duckling_http'
}

In [33]:
interval_dict.keys()

dict_keys(['start', 'end', 'text', 'value', 'confidence', 'additional_info', 'entity', 'extractor'])

In [35]:
interval_dict['additional_info'].keys()

dict_keys(['values', 'to', 'from', 'type'])